<a href="https://colab.research.google.com/github/AdrianDiazGaxiola/Proyecto-Final/blob/main/Precipitaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se importan las librerías a utilizar

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

Se lee el archivo con los datos de registros de Precipitación en México y el archivo con los datos de las temperaturas a nivel nacional

In [57]:
url = "https://github.com/AdrianDiazGaxiola/Proyecto-Final/raw/main/Data/Precipitaci%C3%B3n.csv"
precipitacion = pd.read_csv(url, encoding = 'ISO-8859-1')

url = "https://github.com/AdrianDiazGaxiola/Proyecto-Final/raw/main/Data/Temperaturas%20Completo%20por%20Entidad.csv"
temperaturas = pd.read_csv(url) 

Se combinan en un dataframe los datos de temperaturas con los datos de precipitación

In [66]:
#Como los registros de precipitación son anuales, se utilizarán los registros de temperaturas anuales para futuro análisis.
df = temperaturas.loc[temperaturas['MES']=='ANUAL']

#Se guardan en un dataframe los datos de temperaturas de los años de los cuales se tienen registros se tienen registros de precipitación.
df1 = df.loc[df['ANIO']==2000]
df2 = df.loc[df['ANIO']>=2005]
df = df1.append(df2)
df = df.reset_index(drop=True)

#Se guardan en una lista los años de los cuales se tienen registros de precipitación en México.
Anios = list(range(2000, 2021))
del Anios[1:5]

#Se guardan las abreviaturas de los estados en una lista
estados_abv = ['AGU', 'BCN', 'BCS', 'CAM', 'CHP', 'CHH', 'CMX', 'COA', 'COL', 'DUR', 'GUA', 'GRO', 'HID', 'JAL', 'MEX', 'MIC', 'MOR', 'NAY', 'NLE', 'OAX', 'PUE', 'QUE', 'ROO', 'SLP', 'SIN', 'SON', 'TAB', 'TAM', 'TLA', 'VER', 'YUC', 'ZAC', 'NAC']

#Se crea una lista para guardar en ella los datos de precipitacion para después adjuntarla al dataframe
lluvias = []

for i in Anios:
  l = 0
  for j in estados_abv:
    lluvia = precipitacion.at[l, str(i)] #Se copia el valor del registro de precipitación del estado y año correspondiente
    lluvias.append(lluvia)
    l += 1

df['Precipitacion'] = lluvias

print(df.head())
print(precipitacion.head())
print(df.tail())
print(precipitacion.tail())

   ANIO    MES ENTIDAD      T_Max      T_Min      T_Med Precipitacion
0  2000  ANUAL     AGU  26.932250   8.496315  17.714283           448
1  2000  ANUAL     BCN  26.933117  10.687024  18.810071           203
2  2000  ANUAL     BCS  30.597415  13.695902  22.146658           176
3  2000  ANUAL     CAM  32.666320  19.092887  25.879604         1,138
4  2000  ANUAL     CHP  28.563013  12.150714  20.356864           316
    Entidad federativa   2000   2005   2006  ...   2017   2018   2019   2020
0       Aguascalientes    448    456    459  ...    473    477    477    477
1      Baja California    203    204    202  ...    200    199    199    198
2  Baja California Sur    176    176    178  ...    183    183    184    183
3             Campeche  1,138  1,169  1,175  ...  1,198  1,201  1,202  1,209
4             Coahuila    316    327    327  ...    341    342    341    340

[5 rows x 18 columns]
     ANIO    MES ENTIDAD  T_Max  T_Min  T_Med Precipitacion
556  2020  ANUAL     TLA   23.5    